In [12]:
!%pip install --upgrade langchain
!%pip install --upgrade langchain-openai
!%pip install -qU langchain-core langchain-openai
!%pip install --upgrade openai
!pip install langchainhub

/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control


/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control


In [13]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")







This lab exercise will use the following values:
Azure OpenAI Endpoint: https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/
Azure AI Search: srch-mtpchatbot-dev-frcent-001


In [14]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)

In [15]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

from azure.search.documents.models import (
    VectorizedQuery
)

from langchain_community.vectorstores.azuresearch import AzureSearch

from langchain_core.tools import tool


from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field


In [16]:
class TruckLiveStatus(BaseModel):

    @tool
    def get_trucklive_status(vin: int):
        """Return the TruckLive status of a truck given the vehicle identification number (VIN)."""
        return "activated"


In [17]:
print(get_trucklive_status.name)
print(get_trucklive_status.description)
print(get_trucklive_status.args)
get_trucklive_status.invoke({"vin":1256789})

get_trucklive_status
get_trucklive_status(vin: int) - Return the TruckLive status of a truck given the vehicle identification number (VIN).
{'vin': {'title': 'Vin', 'type': 'integer'}}


'activated'

In [18]:
index_name: str = "mtp-chatbot-hybrid-search-plus-weblate-plus-websites"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=azure_openai_embeddings.embed_query,
)

In [19]:
# Perform a similarity search
results = vector_store.similarity_search(
    query="Wie schließe ich einen TruckLive Vertrag ab?",
    k=3,
    search_type="hybrid",
)
print(results[0].page_content)

1

Schritt 1: 
Registrieren Sie sich für MyTruckPoint. 
Anschließend erscheint eine Schaltfläche zur
Registrierung von TruckLive. 
Um den Prozess zu beginnen, klicken Sie bitte
auf „Weiter“. (     )1

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



2

Schritt 2:
Stimmen Sie dem Daimler Rahmenvertrag
für digitale Dienste zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )3

2

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



3

Schritt 3:
Stimmen Sie dem TruckLive Vertrag zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )

4

5

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



4

Schritt 4:
Definieren Sie einen Zeitpunkt, ab dem TruckLive
für zukünftige Fahrzeuge aktiviert werden soll. (     )
Wählen Sie danach eine Kontaktperson aus dem
eigenen Unternehmen aus, welche die Meldungen 
zu TruckLive bekommen soll. (     ) 
Abschließend klicken Sie auf „Bestätigen“. (     )

Herzlichen Glückwunsch! 
TruckLive ist nun aktiviert und all Ihre Neufahrzeuge
e

In [20]:
# Bringing it together: implement RAG


azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

azure_openai = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)




model_with_tools = azure_openai.bind_tools([TruckLiveStatus], tool_choice="auto")



#model_with_tools = azure_openai.bind_tools([get_trucklive_status], tool_choice="auto")
#model_with_tools.kwargs["tools"]

# Ask the question
query = "What is the TruckLive Status of my truck with the vin 123456789?"

# Create a prompt template with variables, note the curly braces
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["original_question","search_results"],
    template="""
    Question: {original_question}

    Nutze keine anderen Daten, als die zur Verfügung gestellten Daten, für deine Antworten.
    Du bist ein Chatbot für das Kundenportal von Mercedes-Benz Trucks. 
    Antworte freundlich und inkludiere so viele Referenzen wie möglich in deine Anworten.
    Antworte immer in der Sprache, in welcher die Anfrage gestellt wurde.
    {search_results}
    """,
)



results = list(vector_store.similarity_search(
    query=query,
    k=3,
    search_type="hybrid",
))

chain = model_with_tools
chain.invoke(query)

# Build the Prompt and Execute against the Azure OpenAI to get the completion
#chain = LLMChain(llm=model_with_tools, prompt=prompt, verbose=False)
#response = chain.invoke({"original_question": query, "search_results": results})
#print(response)

TypeError: Object of type 'ModelMetaclass' is not JSON serializable